In [1]:
import numpy as np
import pandas as pd

import decouple
import sqlalchemy

In [2]:
#Using enviroment variables for security
user = decouple.config("db_user_mysql")
host = decouple.config('db_host_mysql')
password = decouple.config('db_password_mysql')
database = decouple.config("db_database_mysql")
connection = sqlalchemy.create_engine("mysql+mysqldb://"+user+":"+password+"@"+host+"/"+database)
# sudo apt-get install libmysqlclient-dev

In [3]:
df = pd.read_sql_table('text01', con = connection)
df.head(5).T

,0,1,2,3,4
counter,1,2,3,4,5
paciente,353,341,237,0,245
dataanamnese,2016-12-05 02:45:48,2020-07-15 18:06:50,2017-10-28 20:56:46,2021-03-06 11:38:12,2020-10-04 22:11:17
palavraschave,Voluptate recusandae porro velit harum qui.,None,None,None,None
textoanamnese,<h2>Revisão de Sistemas</h2>\nEos quidem accus...,<h2>Análise de parâmetros clínicos</h2>\nIncid...,<h2>História Patológica Pregressa (HPP)</h2>\n...,<h2>Hábitos de vida</h2>\nLaborum perspiciatis...,<h2>Alergias</h2>\nFugit id repudiandae nemo. ...
diagnostico1,Quisquam tempore ipsa odit.,None,Fugit architecto officia in.,Labore eum sit omnis quos libero.,Deserunt voluptate aliquid aut.
diagnostico2,None,None,None,Dolore corrupti consequatur.,None
timestamp,2021-07-01 18:06:12,2021-07-01 18:06:12,2021-07-01 18:06:12,2021-07-01 18:06:12,2021-07-01 18:06:12
cloud,1,0,1,1,0
update001,2021-02-24 21:08:53,2020-12-29 11:46:53,2021-04-17 12:53:00,2020-12-21 10:51:20,2020-11-01 14:04:58


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   counter        440 non-null    int64         
 1   paciente       440 non-null    int64         
 2   dataanamnese   440 non-null    datetime64[ns]
 3   palavraschave  307 non-null    object        
 4   textoanamnese  440 non-null    object        
 5   diagnostico1   398 non-null    object        
 6   diagnostico2   234 non-null    object        
 7   timestamp      440 non-null    datetime64[ns]
 8   cloud          440 non-null    int64         
 9   update001      440 non-null    datetime64[ns]
 10  ch3            440 non-null    int64         
 11  sysuser        436 non-null    float64       
dtypes: datetime64[ns](3), float64(1), int64(4), object(4)
memory usage: 41.4+ KB


In [5]:
# Colocando o nome das colunas do arquivo de acordo a documentação da iClinic
df = df.rename(
    columns = {
        "paciente":"patient_id",
        "sysuser": "physician_id",
        "dataanamnese": "date",
        "timestamp":"start_time", 
        "update001":"end_time", 
        "textoanamnese":"eventblock_text",
    }
)

In [6]:
df['patient_name'] = np.nan #tratar depois
df['tab_name'] = np.nan 
df['eventblock_name'] = np.nan 

In [7]:
# Tratamento 'patient_name'
df_patient_names = pd.read_csv('desafio-base2-output/patient.csv', index_col=["patient_id"], usecols = ["patient_id","name"])
df_patient_names.head(7).T

patient_id,1,2,3,4,5,6,7
name,Nicolas Freitas Teste,Gustavo Silveira Teste,Otávio Barros Teste,Davi Lucas Fogaça Teste,Bruna Santos Teste,Daniel Campos Teste,João Pedro Lima Teste


In [8]:
# Tratamento 'patient_name'
# teste = df['patient_id'].astype(str).str.contains(r'^427')
# print(teste.value_counts())
# teste = df['patient_id'].values
# print(teste.max())
# df['patient_id'] = df['patient_id'].replace({np.nan: 10000000})

In [9]:
# Tratamento 'patient_name'
df['patient_id'] = [x+1 if x <= df_patient_names.size else 0 for x in df['patient_id'] ]
index = df['patient_id'].values
df['patient_name'] = [ df_patient_names['name'][i] if i != 0 else np.nan for i in index ]

In [10]:
# Removendo a linha que contem NaN pois ter o 'patient_name' e  'patient_id' é mandatório
df = df.dropna(subset = ['patient_name'])

In [13]:
df["eventblock_text"] = '"'+df["eventblock_text"]+'"'
# Sendo necessário que cada Record no arquivo csv esteja escrito em apenas uma linha,
# será necessário um tratamento adicional

In [14]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","start_time","end_time","tab_name","eventblock_name","eventblock_text"]]

In [15]:
df.to_csv('desafio-base2-output/event_record.csv',index=False, encoding='utf-8')